In [ ]:
# %%
import os
import pprint

from agent_tools import (
    cancel_booking,
    get_available_booking_slots,
    get_booking_by_id,
    get_user_details,
    get_vaccination_history,
    login_with_email_password_and_set_access_token,
    register_and_login_user,
    schedule_vaccination_slot,
)

# testing openai connection
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console
from autogen_core.tools import FunctionTool
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from dotenv import load_dotenv

pp = pprint.PrettyPrinter(indent=4)

## Setup Azure OpenAi client
load_dotenv(dotenv_path="../../.env", override=True)

OPENAI_HOST = os.getenv("OPENAI_HOST", "azure")
AZURE_OPENAI_CHATGPT_DEPLOYMENT = os.getenv("AZURE_OPENAI_CHATGPT_DEPLOYMENT")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
AZURE_OPENAI_CHATGPT_MODEL = os.getenv("AZURE_OPENAI_CHATGPT_MODEL")
AZURE_OPENAI_SERVICE = os.getenv("AZURE_OPENAI_SERVICE")


AZURE_OPENAI_ENDPOINT = os.environ.get("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_MODEL = os.environ.get("AZURE_OPENAI_MODEL")
AZURE_AD_TOKEN_SCOPE = os.environ.get("AZURE_AD_TOKEN_SCOPE")
AZURE_OPENAI_CHATGPT_DEPLOYMENT_VERSION = os.environ.get(
    "AZURE_OPENAI_CHATGPT_DEPLOYMENT_VERSION"
)
AZURE_STORAGE_ACCOUNT = os.environ.get("AZURE_STORAGE_ACCOUNT")
AZURE_KEY_VAULT = os.environ.get("AZURE_KEY_VAULT")
AZURE_URL = os.environ.get("AZURE_URL")
SECRET_NAME = os.environ.get("SECRET_NAME")
AZURE_STORAGE_SAS_TOKEN = os.environ.get("AZURE_STORAGE_SAS_TOKEN")
AZURE_SEARCH_INDEX = os.environ.get("AZURE_SEARCH_INDEX")
AZURE_SEARCH_SERVICE = os.environ.get("AZURE_SEARCH_SERVICE")
AZURE_SEARCH_QUERY_LANGUAGE = os.environ.get("AZURE_SEARCH_QUERY_LANGUAGE")
AZURE_SEARCH_QUERY_SPELLER = os.environ.get("AZURE_SEARCH_QUERY_SPELLER")

# check all are not None
assert all(
    [
        AZURE_OPENAI_ENDPOINT,
        AZURE_OPENAI_MODEL,
        AZURE_AD_TOKEN_SCOPE,
        AZURE_OPENAI_CHATGPT_DEPLOYMENT_VERSION,
        AZURE_OPENAI_API_VERSION,
        AZURE_OPENAI_CHATGPT_MODEL,
        AZURE_OPENAI_SERVICE,
        AZURE_OPENAI_CHATGPT_DEPLOYMENT,
        AZURE_STORAGE_ACCOUNT,
        AZURE_KEY_VAULT,
        AZURE_URL,
        SECRET_NAME,
        AZURE_STORAGE_SAS_TOKEN,
        AZURE_SEARCH_INDEX,
        AZURE_SEARCH_SERVICE,
        AZURE_SEARCH_QUERY_LANGUAGE,
        AZURE_SEARCH_QUERY_SPELLER,
    ]
)


# CHATGPT_TOKEN_LIMIT = get_token_limit(AZURE_OPENAI_CHATGPT_MODEL)

## Setup Azure OpenAi client
azure_credential = DefaultAzureCredential(logging_enable=True)

token_provider = get_bearer_token_provider(
    azure_credential, "https://cognitiveservices.azure.com/.default"
)

autogen_openai_client = AzureOpenAIChatCompletionClient(
    azure_deployment=AZURE_OPENAI_CHATGPT_DEPLOYMENT,
    model=AZURE_OPENAI_CHATGPT_MODEL,
    api_version=AZURE_OPENAI_API_VERSION,
    azure_endpoint=f"https://{AZURE_OPENAI_SERVICE}.openai.azure.com",
    azure_ad_token_provider=token_provider,
)

# %%


# os.environ.clear()
os.environ

# %%


async def get_weather(city: str) -> str:
    """Get the weather for a given city."""
    return f"The weather in {city} is 73 degrees and Sunny."


# Define an AssistantAgent with the model, tool, system message, and reflection enabled.
# The system message instructs the agent via natural language.
agent = AssistantAgent(
    name="weather_agent",
    model_client=autogen_openai_client,
    tools=[get_weather],
    system_message="You are a helpful assistant.",
    reflect_on_tool_use=True,
    model_client_stream=True,  # Enable streaming tokens from the model client.
)


# Run the agent and stream the messages to the console.
async def main_test() -> None:
    await Console(agent.run_stream(task="What is the weather in New York?"))


# NOTE: if running this inside a Python script you'll need to use asyncio.run(main()).
# await main_test()

# %% [markdown]
# # FunctionTool declaration

# %%
os.environ.clear()

# %%
load_dotenv(dotenv_path="../../.env", override=True)
BACKEND_DB_URL = os.getenv("BACKEND_DB_URL")
print(BACKEND_DB_URL)
os.getenv("AUTH_TOKEN")

# %% [markdown]
# # Testing: Backend API calling

# %% [markdown]
# ### Register and login immediately

# %%
# load_dotenv(override=True)
BACKEND_DB_URL = os.getenv("BACKEND_DB_URL")

user_data = {
    "nric": "T7636321F",
    "first_name": "tim",
    "last_name": "tom",
    "email": "timm.tom@example.com",
    "date_of_birth": "1990-01-01",
    "gender": "M",
    "postal_code": "123456",
    "password": "Password123",
    "password_confirm": "Password123",
}

# Example usage:
if "AUTH_TOKEN" not in os.environ:
    register_and_login_user(user_data)
else:
    auth_token = os.getenv("AUTH_TOKEN")
    print("Access Token exists and retrieved from environment:", auth_token)
    # print("Access Token:", auth_token)

# %%


# %%
# load_dotenv(dotenv_path="../../.env", override=True)
# os.environ
# os.getenv("AUTH_TOKEN")
# Safely remove the environment variable if it exists
# os.environ.pop("AUTH_TOKEN", None)

# %% [markdown]
# ### Login using email and password

# %%
pp.pprint(
    login_with_email_password_and_set_access_token(
        "xxx@example.com", "xxxx", verbose=True
    )
)
print()

# response = login_with_email_password_and_set_access_token(
#     "timm.tom@example.com", "Password123", verbose=True
# )

response = login_with_email_password_and_set_access_token(
    "kimberly.garza@example.net", "abc123", verbose=True
)


# auth_token = response["access_token"]
# os.environ["AUTH_TOKEN"] = auth_token
pp.pprint(response)

# %% [markdown]
# ### Get user data

# %%
# Call the function to fetch user details
# uses auth_token = os.getenv("AUTH_TOKEN"), successful means the token is valid
pp.pprint(get_user_details())

# %% [markdown]
# ### Get (empty) vaccination history

# %%
response = get_vaccination_history(verbose=True)
pp.pprint(response)

# %% [markdown]
# ### Get booking slots vaccines

# %%
# check available booking slots, with user logged in
available_vac_dummy = [
    "Influenza (INF)",
    "Pneumococcal Conjugate (PCV13)",
    "Human Papillomavirus (HPV)",
    "Tetanus, Diphtheria, Pertussis (Tdap)",
    "Hepatitis B (HepB)",
    "Measles, Mumps, Rubella (MMR)",
    "Varicella (VAR)",
]

get_available_booking_slots("xxx", 2, 2, verbose=True)
response = get_available_booking_slots("Influenza (INF)", 2, 2)
response

# %%
dummy_booking_id = [
    "f804bc1d-3356-4ee3-995b-c0145e4e8ea3",
    "2e691551-f218-48ed-9a82-1ad7c74094ae",
    "40102b01-9676-4460-97fd-1237bdc6859c",
    "3b0ef842-d1f6-49e5-a6c3-6b134c59d744",
    "a6110bfb-d47e-4a7a-afe4-f61fd93fbb23",
]
get_booking_by_id("xx", verbose=True)  # no such booking id
get_booking_by_id(dummy_booking_id[0])  # valid booking id

# %% [markdown]
# ### Schedule and get vaccine records (booked & completed slots)

# %%
# valid booking id
# the return id is vaccination record, which is used for booking cancellation
schedule_vaccination_slot(dummy_booking_id[0], verbose=True)
schedule_vaccination_slot(dummy_booking_id[1], verbose=True)

# %%
# rebook booked slot & invalid booking slot id
print(schedule_vaccination_slot(dummy_booking_id[0], verbose=True))
print(schedule_vaccination_slot("xxxxxx", verbose=True))

# %%
# show two bookings for the logged in user
response = get_vaccination_history()
# vac_record_id_1 = response[0]["id"]
# vac_record_id_2 = response[1]["id"]
response

# %% [markdown]
# ### Cancel booking

# %%
# invalid vaccine record id
cancel_booking("xxx", verbose=True)  # no such booking id

# %%
# valid vaccine record id
cancel_booking("0285c93e-8ade-4705-aea9-8abc177604dd", verbose=True)

# %% [markdown]
# ### Get accine record

# %%
response = get_vaccination_history(verbose=True)
response

# %% [markdown]
# ## Tool wrapping

# %%


def fetch_vaccination_history():
    return "Temp: You have received the following vaccinations: Influenza, Hepatitis A, Hepatitis B, Tetanus, and HPV, but not Covid-19"


def fetch_user_profile():
    return "Temp: Your age is 20, gender male"


def recommend_vaccines():
    return "Temp: I recommend that you get the COVID-19 booster shot."


def check_available_slots():
    return "Temp: There is no available slots at the moment at Clementi Polyclinic. \n\
        But there is available slots for Covid-19 vaccination at Bukit Batok Polyclinic on 10 March 2025, 3:00pm and 4:00pm."


def book_appointment():
    return "Temp: Your appointment has been booked."


# %%
fetch_vaccination_history_tool = FunctionTool(
    fetch_vaccination_history,
    description="Use to retrieve user's vaccination history records based on user id.",
)
fetch_user_profile_tool = FunctionTool(
    fetch_user_profile,
    description="Use to retrieve user profile information such as gender and date of birth based on user id.",
)
recommend_vaccines_tool = FunctionTool(
    recommend_vaccines,
    description="Provide personalised vaccine recommendations based on user's vaccination history, age and gender.",
)
check_slots_tool = FunctionTool(
    check_available_slots,
    description="Check for available vaccination appointment slots based on vaccine name, polyclinic name and date.",
)
book_appointment_tool = FunctionTool(
    book_appointment,
    description="User to book, cancel or reschedule a vaccination appointment.",
)

# These tools can be passed to an agent system to be executed or used by other agents.
# description parameter provides context for how the tool should be used.

# %% [markdown]
# ## Dummy Tool testing

# %%
# register_and_login_user()
# print(os.get)

tools = [
    fetch_vaccination_history_tool,
    fetch_user_profile_tool,
    recommend_vaccines_tool,
    check_slots_tool,
    book_appointment_tool,
]

agent = AssistantAgent(
    name="vaccine_agent",
    model_client=autogen_openai_client,
    tools=tools,
    system_message="You are a helpful assistant for managing vaccination records and appointments.",
    reflect_on_tool_use=True,
    model_client_stream=True,
)


async def test_all_tools():
    test_tasks = [
        "Check singpass login status for user.",
        "Show me the vaccination history for user with NRIC S1234567A.",
        "Get the profile of user U12345.",
        "What vaccines would you recommend for user with NRIC S1234567A?",
        "Are there available slots for COVID-19 at Hougang Polyclinic on 2025-04-01?",
        "Book an appointment for user U12345 to get a Covid-19 vaccination at Bukit Batok Polyclinic on 2025-03-10.",
    ]

    for task in test_tasks:
        print(f"\n🧪 Running task: {task}")
        await Console(agent.run_stream(task=task))


# nest_asyncio.apply()
# asyncio.run(test_all_tools())